In [ ]:
!pip install gspread

In [ ]:
!pip install gspread_pandas

In [ ]:
!pip install beautifulsoup4

In [ ]:
!pip install oauth2client

In [ ]:
!pip install openpyxl

In [1]:
import pandas as pd
import numpy as np
from gspread_pandas import Spread, Client
import gspread_pandas
import gspread
from bs4 import BeautifulSoup
import os
import sys
import re
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('google_secret.json', scope)
gc = gspread.authorize(credentials)

In [5]:
sys.path.append('./')
c = gspread_pandas.conf.get_config(conf_dir='./', file_name='google_secret.json')

In [6]:
def text_processing(text):
    return BeautifulSoup(text.replace('&#xA;', ' '), "html.parser") \
        .get_text(strip=False).replace('\r', ' ').replace('\n',' ')

In [7]:
def find_tags(tags, data, col_name):
    data[col_name] = ''
    for col in data.columns:
        if col.startswith(tags):
            data[col_name] += data[col].fillna(' ') + ','
    data[col_name] = data[col_name].str.replace(' ,', '').str.strip(',')
    # data[col_name] = data[col_name].apply(lambda x: np.nan if x == '' else x)

In [8]:
for file in os.walk(os.getcwd()):
    data_files = [f for f in file[2] if f.startswith('YouScan_Mentions_')]
    break
data_files

['YouScan_Mentions_Карусель_01022022-01022022_c16a6.xlsx',
 'YouScan_Mentions_Перекр_сток_01022022_01022022_925a8.xlsx',
 'YouScan_Mentions_Пятерочка_01022022-01022022_9a6b4.xlsx',
 'YouScan_Mentions_х5_01022022-01022022_25977.xlsx']

In [10]:
for data_file in data_files:
    in_df = pd.read_excel(data_file, engine='openpyxl')
    need_tags = tuple(col for col in in_df.columns if re.match('\d', col, flags=0) or re.match('z', col, flags=0))
    
    in_df['Сумма всех реакций'] = (in_df['Сумма всех реакций'].fillna(0) +
                                in_df['Репосты'].fillna(0) +
                                in_df['Комментарии'].fillna(0))
    
    in_df['Дайджест текста'] = (in_df['Дайджест текста'].fillna('')
                                            .apply(text_processing))
    in_df['Автор'] = (in_df['Автор'].fillna('')
                      .apply(text_processing))
    in_df['Место публикации'] = (in_df['Место публикации'].fillna('')
                                             .apply(text_processing))
    
    find_tags(need_tags, in_df, 'all_tags')
    
    out_df = in_df[['URL', 'Дайджест текста', 'Источник', 'Подписчики', 
                'Город', 'Регион', 'Автор', 'Место публикации', 'Профиль', 
                'Подписчики места публикации', 'Профиль места публикации', 
                'Тональность', 'Тип поста', 'Сумма всех реакций', 'Дата', 'all_tags']]
    
    frame_columns = ['tag_' + str(x) for x in range(out_df.all_tags.apply(lambda x: len(x.split(','))).max())]
    tags_df = pd.DataFrame(out_df.all_tags.str.split(',').to_list(), columns=frame_columns)
    out_df = pd.concat([out_df, tags_df], axis=1)
    
    out_df[['Подписчики', 
            'Подписчики места публикации', 
            'Сумма всех реакций']] = out_df[['Подписчики', 
                                             'Подписчики места публикации', 
                                             'Сумма всех реакций']].fillna(0).astype(int)
    
    if 'х5' in data_file:
        sht = gc.open("Копия X5 Рабочий файл")
        sht.values_clear("Data!A1:X20000")
        spread = Spread('Копия X5 Рабочий файл', config=c)
        spread.df_to_sheet(out_df, sheet='Data', start='A1', index=False)
        print(f'Upload {data_file}')
    elif 'Карусель' in data_file:
        sht = gc.open("Копия Карусель Рабочий файл")
        sht.values_clear("Data!A1:X20000")
        spread = Spread('Копия Карусель Рабочий файл', config=c)
        spread.df_to_sheet(out_df, sheet='Data', start='A1', index=False)
        print(f'Upload {data_file}')
    elif 'Перекр_сток' in data_file:
        sht = gc.open("Копия Перекресток Рабочий файл")
        sht.values_clear("Data!A1:X20000")
        spread = Spread('Копия Перекресток Рабочий файл', config=c)
        spread.df_to_sheet(out_df, sheet='Data', start='A1', index=False)
        print(f'Upload {data_file}')
    elif 'Пятерочка' in data_file:
        sht = gc.open("Копия Пятерочка Рабочий файл")
        sht.values_clear("Data!A1:X20000")
        spread = Spread('Копия Пятерочка Рабочий файл', config=c)
        spread.df_to_sheet(out_df, sheet='Data', start='A1', index=False)
        print(f'Upload {data_file}')
    elif 'Чижик' in data_file:
        sht = gc.open("Копия Чижик Рабочий файл")
        sht.values_clear("Data!A1:X20000")
        spread = Spread('Копия Чижик Рабочий файл', config=c)
        spread.df_to_sheet(out_df, sheet='Data', start='A1', index=False)
        print(f'Upload {data_file}')
    else:
        print('ERROR!!!')

Upload YouScan_Mentions_Карусель_01022022-01022022_c16a6.xlsx
Upload YouScan_Mentions_Перекр_сток_01022022_01022022_925a8.xlsx
Upload YouScan_Mentions_Пятерочка_01022022-01022022_9a6b4.xlsx
Upload YouScan_Mentions_х5_01022022-01022022_25977.xlsx
